# <b>1. Model setup</b>

In [ ]:
import os

# Download model weight
if not os.path.exists('yolov7.pt'):
  # Download model weight
  !curl -L -o yolov7.pt https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt

# Create the model
from detect_isu import ObjectDetector
detector = ObjectDetector()
detector.set_conf('weights', 'yolov7.pt')
detector.setup_model()

# <b>2. Test on one image</b>

## 2.1 Read and display the image

In [ ]:
import cv2
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (40,10)

# read the image
img = cv2.imread('inference/images/vehicle_test.png')

# display the image
# cv2.imread: BGR -> plt.imshow(rgb)
plt.imshow(img[:, :, ::-1])
plt.title('Input image')
plt.axis('off')
plt.show()

## 2.2 Run vehicle detection

In [ ]:
output_box = detector.detect(img, ['truck', 'bus', 'car'], do_visual=False)
print(f"detect {output_box.shape[0]} objects")
print("The output is a Nx6 matrix")
print("Each row is one detection: box coordinates, prediction confidence, class_id")
print('--------')
print("Let's look at the first detection")
print(output_box[0])

# 2.3 Plot the detection result on the image

In [ ]:
output_visual = detector.plot_box(img[:, :, ::-1], output_box)
plt.imshow(output_visual)
plt.title('Detetion results')
plt.axis('off')
plt.show()

# <b>3. Test on one video</b>

## 3.1 Display the video

In [ ]:
from IPython.display import HTML
from base64 import b64encode

def show_video(video_path, video_width = 600):   
  video_file = open(video_path, "r+b").read() 
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")
 
video_source = 'inference/videos/vehicle_test.mp4'
show_video(video_source)

## 3.2 Run detection and save result

To save time, run it on the first 100 frames.

In [ ]:
import cv2

cap = cv2.VideoCapture(video_source)
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
output_video = None
output_video_name = 'detection_output.mp4'

frame_id = 0
ret = True
while ret:
    ret, frame = cap.read()        
    if ret:
      print(f'process frame {frame_id}')
      output_visual = detector.detect(frame, ['truck', 'bus', 'car'], verbose=False)      
      if output_video is None:
          frame_height, frame_width = output_visual.shape[:2]
          output_video = cv2.VideoWriter(output_video_name, fourcc, fps, (frame_width, frame_height))
      output_video.write(output_visual)
      frame_id += 1
      if frame_id > 100:
        break

cap.release()
output_video.release()

## 3.3 Visualize result

In [ ]:
import imageio
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML

# show_video doesn't work with opencv output video
# try an alternative way to visualize it
def display_video(video):
    fig = plt.figure(figsize=(40, 10))  #Display size specification
    mov = []
    for i in range(len(video)):  #Append videos one by one to mov
        img = plt.imshow(video[i], animated=True)
        plt.axis('off')
        mov.append([img])
    #Animation creation
    anime = animation.ArtistAnimation(fig, mov, interval=50, repeat_delay=1000)
    plt.close()
    return anime


video = imageio.mimread(output_video_name)  #Loading video
HTML(display_video(video).to_html5_video())  #Inline video display in HTML5